# Build Strands Agents with SageMaker AI Models and MLflow

---

## Overview

This notebook accompanies the blog post **"Build Strands agents with SageMaker AI models and MLflow"** and provides a hands-on guide to building AI agents using the Strands Agents SDK with models deployed on Amazon SageMaker AI endpoints, while leveraging SageMaker Managed MLflow for comprehensive agent observability.

### What You Will Learn

1. **Understand Strands Agents SDK** - Build AI agents with just a few lines of code
2. **Deploy Models on SageMaker AI** - Deploy foundation models from SageMaker JumpStart
3. **Integrate Strands with SageMaker** - Use SageMaker-deployed models with Strands agents
4. **Set Up Agent Observability** - Configure SageMaker Managed MLflow for agent tracing
5. **Implement A/B Testing with Evaluation** - Deploy multiple model variants and evaluate with MLflow metrics
6. **Evaluate Agents with Strands Framework** - Use Strands built-in evaluation for comprehensive testing

### Why Use SageMaker AI Instead of Bedrock?

- **Infrastructure Control**: Full control over compute instances, networking, and scaling
- **Model Flexibility**: Deploy custom models, fine-tuned variants, or open-source alternatives
- **Cost Predictability**: Precise cost forecasting through reserved instances
- **Advanced MLOps**: Enterprise-grade model governance with MLflow integration

**Estimated time: ~75 minutes** | ⚠️ Run cells sequentially from top to bottom

---

## 1. Prerequisites and Setup

Before building our Strands agent, we need to set up our environment with the required packages.

### Prerequisites

- AWS Account with access to Amazon Bedrock and Amazon SageMaker AI
- IAM role with access to SageMaker AI, Bedrock, MLflow, S3, and JumpStart
- Jupyter notebook running locally or on SageMaker AI Studio

In [ ]:
%%writefile requirements.txt
strands-agents>=1.9.1
strands-agents-tools>=0.2.8
mlflow>=3.4.0
strands-agents[sagemaker]
pandas>=2.0.0

In [ ]:
!pip install -r requirements.txt

---

## 2. Building Your First Strands Agent

Strands Agents SDK is an open source SDK that takes a model-driven approach to building and running AI agents. It combines:

- **A Model**: The foundation model that powers the agent's reasoning
- **A System Prompt**: Instructions that guide the agent's behavior
- **Tools**: Capabilities the agent can use to interact with external systems

Let's start with a simple agent using Amazon Bedrock to understand the basics before moving to SageMaker.

In [ ]:
from strands.models.bedrock import BedrockModel
from strands import Agent
from strands_tools import http_request

# Create a model using Amazon Bedrock
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
)

# Create an agent with the model and tools
agent = Agent(model=model, tools=[http_request])

# Test the agent
agent("Where is the international space station now?")

---

## 3. Why Use Models Deployed on SageMaker AI?

Organizations may choose to deploy foundation models on SageMaker AI for several reasons:

| Benefit | Description |
|---------|-------------|
| **Infrastructure Control** | Full control over compute instances, networking configurations, and scaling policies. Crucial for organizations with strict latency SLAs or specific hardware requirements. |
| **Model Flexibility** | Deploy any model - custom architectures, fine-tuned variants, or open-source alternatives like Llama or Mistral. |
| **Cost Predictability** | Precise cost forecasting and optimization through reserved instances, spot pricing, and right-sized compute resources. |
| **Advanced MLOps** | Integration with MLflow, model registry, and A/B testing capabilities for enterprise-grade model governance. |

---

## 4. Deploy Model as SageMaker AI Endpoint

Now we'll deploy a Qwen3-4B model from SageMaker JumpStart as an inference endpoint. SageMaker JumpStart provides pre-trained models that can be deployed with just a few lines of code.

**Note**: Any model you use with Strands Agents SDK should support OpenAI compatible chat completions APIs.

⏱️ **This step takes approximately 5-10 minutes** to complete as the endpoint is being provisioned.

In [ ]:
import boto3, botocore
from boto3.session import Session
from sagemaker.jumpstart.model import JumpStartModel
import sagemaker

boto_session = Session()
sts = boto3.client('sts')
account_id = sts.get_caller_identity().get("Account")
region = boto_session.region_name
role = sagemaker.get_execution_role()

ENDPOINT_NAME = INITIAL_CONFIG_NAME = "llm-qwen-endpoint-sagemaker"

# Check if endpoint already exists
sagemaker_client = boto3.client('sagemaker')

try:
    endpoint_info = sagemaker_client.describe_endpoint(EndpointName=ENDPOINT_NAME)
    endpoint_status = endpoint_info['EndpointStatus']
    
    if endpoint_status == 'InService':
        print(f"✅ Endpoint '{ENDPOINT_NAME}' already exists and is InService. Reusing existing endpoint.")
    else:
        print(f"⏳ Endpoint '{ENDPOINT_NAME}' exists with status: {endpoint_status}. Waiting...")
        waiter = sagemaker_client.get_waiter('endpoint_in_service')
        waiter.wait(EndpointName=ENDPOINT_NAME)
        print(f"✅ Endpoint '{ENDPOINT_NAME}' is now InService.")

except (sagemaker_client.exceptions.ResourceNotFound, botocore.exceptions.ClientError):
    # Endpoint doesn't exist, create it
    print(f"Endpoint '{ENDPOINT_NAME}' not found. Creating new endpoint...")
    
    model_a = JumpStartModel(
        model_id="huggingface-reasoning-qwen3-4b", 
        model_version="1.0.0",
        name="qwen3-4b-model"
    )
    
    predictor_a = model_a.deploy(
        initial_instance_count=1,
        instance_type="ml.g5.2xlarge",
        endpoint_name=ENDPOINT_NAME
    )
    
    print(f"✅ Endpoint '{ENDPOINT_NAME}' deployed successfully!")


---

## 5. Building a Strands Agent with SageMaker AI Models

With the model deployed, we can now create a `SageMakerAIModel` and use it with Strands Agents. The Strands Agents SDK implements a SageMaker provider that allows you to run agents against models deployed on SageMaker inference endpoints.

Key configuration options:
- **endpoint_name**: The name of your SageMaker endpoint
- **region_name**: AWS region where the endpoint is deployed
- **max_tokens**: Maximum tokens in the response
- **temperature**: Controls randomness (lower = more deterministic)
- **stream**: Enable streaming responses

In [ ]:
from strands.models.sagemaker import SageMakerAIModel
from strands import Agent, tool
from strands_tools import http_request, calculator

model_sagemaker = SageMakerAIModel(
    endpoint_config={
        "endpoint_name": ENDPOINT_NAME,
        "region_name": region
    },
    payload_config={
        "max_tokens": 2048,
        "temperature": 0.2,
        "stream": True,
    }
)

# Test the agent with SageMaker model
agent = Agent(model=model_sagemaker, tools=[http_request], callback_handler=None)
agent("Where is the international space station now? (Use: http://api.open-notify.org/iss-now.json)")

---

## 6. Using SageMaker AI Serverless MLflow App for Agent Observability

SageMaker AI Serverless MLflow provides comprehensive observability for AI agents by:

- **Automatic Trace Capture**: Captures execution traces, tool usage patterns, and decision-making workflows
- **No Custom Instrumentation**: Works out of the box with Strands Agents SDK
- **Centralized Monitoring**: Monitor agent behavior across multiple deployments
- **Audit Trails**: Maintain compliance requirements with detailed execution logs

### Step 1: Create MLflow Tracking Server

First, we'll create a S3 bucket to hold MLflow artifacts and then SageMaker Managed MLflow App using the SageMaker SDK.

In [ ]:
# Create S3 bucket for MLflow artifacts
s3_client = boto3.client('s3', region_name=region)
bucket_name = f'{account_id}-mlflow-bucket'

try:
    # Check if bucket exists
    s3_client.head_bucket(Bucket=bucket_name)
    print(f"✅ S3 bucket '{bucket_name}' already exists.")
except s3_client.exceptions.ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == '404':
        # Bucket doesn't exist, create it
        print(f"Creating S3 bucket '{bucket_name}'...")
        if region == 'us-east-1':
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"✅ S3 bucket '{bucket_name}' created successfully!")
    else:
        raise e

In [ ]:
MLFLOW_APP_NAME = 'strands-mlflow-app'

# Check if MLflow app already exists by listing apps
existing_app = None
try:
    apps_response = sagemaker_client.list_mlflow_apps()
    for app in apps_response.get('Summaries', []):
        if app['Name'] == MLFLOW_APP_NAME:
            existing_app = app
            break
except Exception as e:
    print(f"Error listing apps: {e}")

if existing_app:
    # App exists, get details
    mlflow_app_details = sagemaker_client.describe_mlflow_app(Arn=existing_app['Arn'])
    status = mlflow_app_details['Status']
    
    if status == 'RUNNING':
        print(f"✅ MLflow app '{MLFLOW_APP_NAME}' already exists and is running.")
    else:
        print(f"⏳ MLflow app '{MLFLOW_APP_NAME}' exists with status: {status}")
    
    print(f"   ARN: {existing_app['Arn']}")
    mlflow_app_details = {'Arn': existing_app['Arn']}
else:
    # MLflow app doesn't exist, create it
    print(f"MLflow app '{MLFLOW_APP_NAME}' not found. Creating new app...")
    
    mlflow_app_details = sagemaker_client.create_mlflow_app(
        Name=MLFLOW_APP_NAME,
        ArtifactStoreUri=f's3://{account_id}-mlflow-bucket/artifacts',
        RoleArn=role,
    )
    
    print(f"✅ MLflow app creation initiated: {mlflow_app_details['Arn']}")

MLflow app takes few minutes to be ready for use. Check the status of the app and continue once status is "created"

In [ ]:
import time

def wait_for_mlflow_app(sagemaker_client, arn, timeout=600, poll_interval=30):
    """Wait for MLflow app to be ready."""
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        app_info = sagemaker_client.describe_mlflow_app(Arn=arn)
        status = app_info['Status']
        
        print(f"MLflow app status: {status}")
        
        if status in ['Created','Updated']:
            print("✅ MLflow app is 'Created' and ready for use!")
            return app_info
        elif status in ['Failed', 'Deleted']:
            raise Exception(f"MLflow app failed with status: {status}")
        
        time.sleep(poll_interval)
    
    raise TimeoutError(f"MLflow app did not become ready within {timeout} seconds")

# Wait for the app to be ready
app_info = wait_for_mlflow_app(sagemaker_client, mlflow_app_details['Arn'])


### Step 2: Configure MLflow Tracking for Strands Agents

Now we enable automatic logging for Strands agents so that all agent interactions, tool usage, and performance metrics are automatically captured.

In [ ]:
import os
import mlflow

tracking_uri = mlflow_app_details['Arn']
print(f"MLflow App URL: {tracking_uri}")

# Set MLflow tracking URI
os.environ["MLFLOW_TRACKING_URI"] = tracking_uri
# Or you can set the tracking server as below:
# mlflow.set_tracking_uri(tracking_uri)

# Set experiment name and enable auto-logging
mlflow.set_experiment("Strands-MLflow")
mlflow.strands.autolog()

print("MLflow tracking configured successfully!")

### Step 3: Run the Agent with Tracing Enabled

With MLflow tracking configured and auto-logging enabled, we can now run our Strands Agent. All traces will be automatically captured.

In [ ]:
def capitalize(response):
    return response.upper()

agent = Agent(model=model_sagemaker, tools=[http_request])
agent_response = agent("Where is the international space station now? (Use: http://api.open-notify.org/iss-now.json)")
capitalize(agent_response.message['content'][0]['text'])

### Step 4: View Traces in MLflow UI

After running the agent, traces and metrics are available in the MLflow App. You can access MLflow UI using the presigned URL below. Go to "Strands-MLflow" under expertiments and check on the generated trace. 

In [ ]:
# Get presigned URL for MLflow tracking server
presigned_response = sagemaker_client.create_presigned_mlflow_app_url(
    Arn=mlflow_app_details['Arn'] 
)

mlflow_ui_url = presigned_response['AuthorizedUrl']
print(f"MLflow UI URL: {mlflow_ui_url}")

Your trace will give you the details of all steps take by the agent to fulfill your request.
<img src="./images/first_agent_trace.png">

### Step 5. Manual Tracing for Complete Visibility

While MLflow's automatic tracing captures agent invocations and tool calls, other function calls (like our `capitalize` function) are not logged automatically.

To capture the complete execution flow, we can use MLflow's manual tracing capability with the `@mlflow.trace` decorator.

In [ ]:
@mlflow.trace(span_type="func", attributes={"operation": "capitalize"})
def capitalize(response):
    return response.upper()

@mlflow.trace
def run_agent():
    agent = Agent(model=model_sagemaker, tools=[http_request])
    mlflow.update_current_trace(request_preview="Run Strands Agent")
    
    agent_response = agent("Where is the international space station now? (Use: http://api.open-notify.org/iss-now.json)")
    capitalized_response = capitalize(agent_response.message['content'][0]['text'])
    
    return capitalized_response

# Execute the traced function
capitalized_response = run_agent()
print(capitalized_response)

---

## 7. Deploying a New LLM for A/B Testing

With Amazon SageMaker AI, you can optimize LLMs for your agent applications through A/B testing. This allows you to:

- Deploy a new model alongside your existing one
- Distribute traffic between both endpoints
- Measure the impact before fully committing to an upgrade

In this section, we'll upgrade from Qwen3-4B to Qwen3-8B using A/B testing.

### Step 1: Deploy the New Model

In [ ]:
# Step 1: Create a model from JumpStart
model_b_name = "qwen3-8b-model"
model_b_id, model_b_version = "qwen3-8b-model", "1.0.0"

model_8b = JumpStartModel(
    model_id="huggingface-reasoning-qwen3-8b",  
    model_version="1.0.0",
    name=model_b_name
)
model_8b.create(instance_type="ml.g5.2xlarge")

print(f"Model '{model_b_name}' created successfully!")

### Step 2: Configure Production Variants for A/B Testing

We'll create production variants with 50/50 traffic split between the champion (4B) and challenger (8B) models.

In [ ]:
# Step 2: Create production variants for A/B testing
production_variants = [
    # The original model (champion)
    {
        "VariantName": "qwen-4b-variant",
        "ModelName": "qwen3-4b-model",
        "InitialInstanceCount": 1,
        "InstanceType": "ml.g5.2xlarge",
        "InitialVariantWeight": 0.5  # 50% of traffic
    },
    # The new model (challenger)
    {
        "VariantName": "qwen-8b-variant",
        "ModelName": model_b_name,
        "InitialInstanceCount": 1,
        "InstanceType": "ml.g5.2xlarge",
        "InitialVariantWeight": 0.5  # 50% of traffic
    }
]

# Step 3: Create new endpoint configuration
ENDPOINT_CONFIG_AB_TESTING = "llm-endpoint-config-ab"
sagemaker_client.create_endpoint_config(
    EndpointConfigName=ENDPOINT_CONFIG_AB_TESTING,
    ProductionVariants=production_variants
)

print(f"Endpoint config '{ENDPOINT_CONFIG_AB_TESTING}' created!")

### Step 3: Update the Endpoint with A/B Testing Configuration

⏱️ **This step takes several minutes** as the endpoint is being updated with the new configuration.

In [ ]:
# Step 4: Update the endpoint with new A/B testing configuration
sagemaker_client.update_endpoint(
    EndpointName=ENDPOINT_NAME,  # The endpoint name stays the same
    EndpointConfigName=ENDPOINT_CONFIG_AB_TESTING
)

# Wait until the update is completed
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
waiter.wait(EndpointName=ENDPOINT_NAME)

print(f"Endpoint '{ENDPOINT_NAME}' updated with A/B testing configuration!")

### Step 4: Create Agents for Each Variant

For controlled experiments, we create separate agents that target specific variants using the `target_variant` parameter.

In [ ]:
from strands.models.sagemaker import SageMakerAIModel
from strands import Agent, tool
from strands_tools import http_request, calculator

# Agent targeting the 4B variant (champion)
model_sagemaker_a = SageMakerAIModel(
    endpoint_config={
        "endpoint_name": ENDPOINT_NAME,
        "region_name": region,
        "target_variant": "qwen-4b-variant"
    },
    payload_config={
        "max_tokens": 2048,
        "temperature": 0.2,
        "stream": True,
    }
)

# Agent targeting the 8B variant (challenger)
model_sagemaker_b = SageMakerAIModel(
    endpoint_config={
        "endpoint_name": ENDPOINT_NAME,
        "region_name": region,
        "target_variant": "qwen-8b-variant"
    },
    payload_config={
        "max_tokens": 2048,
        "temperature": 0.2,
        "stream": True,
    }
)

print("Variant-specific agents created!")

### Step 5: Create Evaluation Dataset

We create a structured evaluation dataset compatible with `mlflow.genai.evaluate()`. Each entry includes:

- **inputs**: The query/prompt for the agent
- **expectations**: Ground truth values including expected tool and expected_facts (for Correctness scorer)

In [ ]:
import pandas as pd
import time

# Define evaluation dataset for MLflow GenAI evaluate
# expected_facts is used by the built-in Correctness scorer (LLM judge)
eval_dataset = [
    {
        "inputs": {"query": "Calculate 15% tip on a $85.50 restaurant bill. Use calculator tool"},
        "expectations": {
            "expected_tool": "calculator",
            "expected_facts": ["The tip amount is $12.825 or approximately $12.82 or $12.83", "15% of 85.50 equals 12.825"]
        }
    },
    {
        "inputs": {"query": "What is 2048 divided by 64? Use calculator tool"},
        "expectations": {
            "expected_tool": "calculator",
            "expected_facts": ["The answer is 32", "2048 divided by 64 equals 32"]
        }
    },
    {
        "inputs": {"query": "Calculate the square root of 144. Use calculator tool"},
        "expectations": {
            "expected_tool": "calculator",
            "expected_facts": ["The square root of 144 is 12"]
        }
    },
    {
        "inputs": {"query": "What is 25 multiplied by 4, then add 10? Use calculator tool"},
        "expectations": {
            "expected_tool": "calculator",
            "expected_facts": ["The answer is 110", "25 times 4 is 100, plus 10 equals 110"]
        }
    },
    {
        "inputs": {"query": "If I have $500 and spend 30%, how much do I have left? Use calculator tool"},
        "expectations": {
            "expected_tool": "calculator",
            "expected_facts": ["$350 remaining", "30% of 500 is 150, so 500 minus 150 equals 350"]
        }
    }
]

print(f"Created evaluation dataset with {len(eval_dataset)} test cases")
print("\nSample test case:")
print(eval_dataset[0])

### Step 6: Define Scorers for MLflow Evaluation

MLflow GenAI evaluation uses **scorers** to assess agent performance. We'll use a combination of custom and built-in scorers:

| Scorer | Type | Description |
|--------|------|-------------|
| **tool_selection_scorer** | Custom | Checks if the agent selected the correct tool for the task |
| **Correctness** | Built-in | LLM judge that evaluates factual correctness using expected_facts |
| **RelevanceToQuery** | Built-in | LLM judge that evaluates if response addresses the query |

In [ ]:
from mlflow.genai.scorers import scorer, Correctness, RelevanceToQuery
from mlflow.entities import Feedback, AssessmentSource, AssessmentSourceType

@scorer
def tool_selection_scorer(inputs: dict, outputs: str, expectations: dict) -> Feedback:
    """
    Evaluates if the agent used the expected tool.
    Checks if the expected tool name appears in the output or trace.
    """
    expected_tool = expectations.get("expected_tool", "")
    
    # Check if tool was mentioned in output (simplified check)
    # In production, you'd parse the trace for actual tool calls
    tool_used = expected_tool.lower() in outputs["tools"] if outputs else False
    
    return Feedback(
        name="tool_selection",
        value=1.0 if tool_used else 0.0,
        rationale=f"Expected tool '{expected_tool}' {'was' if tool_used else 'was NOT'} used",
        source=AssessmentSource(
            source_type=AssessmentSourceType.CODE,
            source_id="tool_selection_scorer_v1"
        )
    )

# Built-in scorers from MLflow GenAI:
# - Correctness: LLM judge that evaluates factual correctness using expected_facts
# - RelevanceToQuery: LLM judge that evaluates if response addresses the query

print("✅ Scorers configured!")
print("   - tool_selection_scorer (custom): Checks if correct tool was used")
print("   - Correctness (built-in): LLM judge for factual correctness")
print("   - RelevanceToQuery (built-in): LLM judge for response relevance")

### Step 7: Run MLflow GenAI Evaluation for Each Agent

We use `mlflow.genai.evaluate()` to run the evaluation for each model variant. This:

1. Executes the agent on each test case
2. Applies all scorers to assess performance
3. Logs results to MLflow for visualization

In [ ]:
import mlflow
from strands import Agent
from strands_tools import calculator

# Set experiment for A/B evaluation
mlflow.set_experiment("Strands_Agents_AB_Evaluation")

# Define predict functions for each agent
def predict_4b(query: str) -> str:
    """Prediction function for Qwen 4B agent"""
    agent_model_a = Agent(model=model_sagemaker_a, tools=[calculator])
    response = agent_model_a(query)
    return {"outputs": str(response), "tools": list(response.metrics.tool_metrics.keys())}

def predict_8b(query: str) -> str:
    """Prediction function for Qwen 8B agent"""
    agent_model_b = Agent(model=model_sagemaker_b, tools=[calculator])
    response = agent_model_b(query)
    return {"outputs": str(response), "tools": list(response.metrics.tool_metrics.keys())}

print("✅ Agents and predict functions ready!")

In [ ]:
# Run evaluation for Qwen 4B variant
print("Evaluating Qwen 4B variant...")
print("=" * 50)

eval_results_4b = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_4b,
    scorers=[
        tool_selection_scorer,
        Correctness(model="bedrock:/us.amazon.nova-pro-v1:0"),
        RelevanceToQuery(model="bedrock:/us.amazon.nova-pro-v1:0")
    ]
)

print(f"\n✅ Qwen 4B evaluation complete!")
print(f"Run ID: {eval_results_4b.run_id}")

In [ ]:
# Run evaluation for Qwen 8B variant
print("Evaluating Qwen 8B variant...")
print("=" * 50)

eval_results_8b = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_8b,
    scorers=[
        tool_selection_scorer,
        Correctness(model="bedrock:/us.amazon.nova-pro-v1:0"),
        RelevanceToQuery(model="bedrock:/us.amazon.nova-pro-v1:0")
    ]
)

print(f"\n✅ Qwen 8B evaluation complete!")
print(f"Run ID: {eval_results_8b.run_id}")

### Step 8: Compare Evaluation Results

Let's compare the performance of both model variants across our evaluation metrics.

In [ ]:
import pandas as pd

# Get metrics from evaluation results
metrics_4b = eval_results_4b.metrics
metrics_8b = eval_results_8b.metrics

# Create comparison dataframe
comparison_data = {
    "Metric": [],
    "Qwen 4B": [],
    "Qwen 8B": [],
    "Winner": []
}

for metric_name in metrics_4b.keys():
    val_4b = metrics_4b.get(metric_name, 0)
    val_8b = metrics_8b.get(metric_name, 0)
    
    comparison_data["Metric"].append(metric_name)
    comparison_data["Qwen 4B"].append(f"{val_4b:.3f}" if isinstance(val_4b, float) else val_4b)
    comparison_data["Qwen 8B"].append(f"{val_8b:.3f}" if isinstance(val_8b, float) else val_8b)
    
    if isinstance(val_4b, (int, float)) and isinstance(val_8b, (int, float)):
        if val_4b > val_8b:
            comparison_data["Winner"].append("4B ✓")
        elif val_8b > val_4b:
            comparison_data["Winner"].append("8B ✓")
        else:
            comparison_data["Winner"].append("Tie")
    else:
        comparison_data["Winner"].append("-")

comparison_df = pd.DataFrame(comparison_data)

print("=" * 70)
print("A/B TESTING EVALUATION RESULTS")
print("=" * 70)
print(comparison_df.to_string(index=False))

In [ ]:
import mlflow
from IPython.display import display, Markdown

# Get presigned URL
response = sagemaker_client.create_presigned_mlflow_app_url(
    Arn=mlflow_app_details['Arn'],
    ExpiresInSeconds=300
)
presigned_url = response['AuthorizedUrl']

# Get run info
run_4b = mlflow.get_run(eval_results_4b.run_id)
run_8b = mlflow.get_run(eval_results_8b.run_id)

display(Markdown(f"""
## 📊 View A/B Evaluation Comparison in MLflow

🔗 **[Open MLflow UI]({presigned_url})**

Once authenticated, select the **Evaluations** tab and compare these runs:

| Run Name | Run ID |
|----------|--------|
| {run_4b.info.run_name} | `{eval_results_4b.run_id[:8]}...` |
| {run_8b.info.run_name} | `{eval_results_8b.run_id[:8]}...` |
"""))


<img src="./images/mlflow_eval_compare.png"/>

### Step 9: Transition to the New Model (Optional)

Based on the evaluation results, if the new model performs better, you can transition by adjusting the variant weights.

In [ ]:
# Uncomment to transition fully to the 8B model if it performs better
"""
production_variants = [
    {
        "VariantName": "qwen-8b-variant",
        "ModelName": model_b_name,
        "InitialInstanceCount": 1,
        "InstanceType": "ml.g5.2xlarge",
        "InitialVariantWeight": 1
    }
]

ENDPOINT_CONFIG_QWEN3_8b = "llm-endpoint-config-qwen3-8b"
sagemaker_client.create_endpoint_config(
    EndpointConfigName=ENDPOINT_CONFIG_QWEN3_8b,
    ProductionVariants=production_variants
)
sagemaker_client.update_endpoint(
    EndpointName=ENDPOINT_NAME,  # The endpoint name stays the same
    EndpointConfigName=ENDPOINT_CONFIG_QWEN3_8b
)

# Wait until the update is completed
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
waiter.wait(EndpointName=ENDPOINT_NAME)

# validate that new model with a Strands agent. 
model_sagemaker = SageMakerAIModel(
    endpoint_config={
        "endpoint_name": ENDPOINT_NAME,
        "region_name": region
    },
    payload_config={
        "max_tokens": 2048,
        "temperature": 0.2,
        "stream": True,
    }
)

# Test the agent with SageMaker model
agent = Agent(model=model_sagemaker, tools=[http_request], callback_handler=None)
agent("Where is the international space station now? (Use: http://api.open-notify.org/iss-now.json)")
"""

---

## 10. Troubleshooting

### Common Issues with MLflow Tracing

If you encounter `ImportError: cannot import name 'TokenUsageKey' from 'mlflow.tracing.constant'` or other tracing issues:

1. **Check MLflow version**: Should be 3.4.0 or greater
2. **Verify IAM permissions**: Your role needs access to:
   - Read, write, list the S3 bucket used as the artifact location
   - Access MLflow tracking server

### Verify MLflow Version

In [ ]:
import mlflow
print(f"MLflow version: {mlflow.__version__}")
assert mlflow.__version__ >= "3.4.0", "Please upgrade MLflow to version 3.4.0 or greater"

---

## 11. Cleanup

⚠️ **Important**: Run this section to delete the resources created in this notebook and avoid ongoing charges.

This will delete:
- SageMaker endpoint
- Endpoint configurations
- MLflow tracking server
- Local evaluation files

In [ ]:
# Uncomment  to delete created resources
"""import os

# Delete the endpoint
sagemaker_client.delete_endpoint(EndpointName=ENDPOINT_NAME)
print(f"Deleted endpoint: {ENDPOINT_NAME}")

# Delete endpoint configurations
sagemaker_client.delete_endpoint_config(EndpointConfigName=INITIAL_CONFIG_NAME)
print(f"Deleted endpoint config: {INITIAL_CONFIG_NAME}")

sagemaker_client.delete_endpoint_config(EndpointConfigName=ENDPOINT_CONFIG_AB_TESTING)
print(f"Deleted endpoint config: {ENDPOINT_CONFIG_AB_TESTING}")

# Delete MLflow tracking server
sagemaker_client.delete_mlflow_app(
    Arn=mlflow_app_details["Arn"]
)
print(f"Deleted MLflow app: {mlflow_app_details['Arn']}")

print("\n✅ Cleanup completed successfully!")"""

---

## Conclusion

In this notebook, we explored how to build AI agents using Strands Agents SDK with models deployed on Amazon SageMaker AI endpoints, while leveraging SageMaker Managed MLflow for comprehensive agent observability and evaluation.

### What We Accomplished

1. **Built a basic Strands Agent** using Amazon Bedrock as the model provider
2. **Deployed a Qwen model** from SageMaker JumpStart as an inference endpoint
3. **Integrated SageMaker models** with Strands Agents for greater infrastructure control
4. **Set up MLflow observability** for automatic agent tracing and monitoring
5. **Implemented A/B testing with evaluation** comparing model variants using:
   - Tool Selection Accuracy
   - Task Completion Rate
   - Response Latency

### Key Evaluation Metrics

| Metric | Description | Use Case |
|--------|-------------|----------|
| Tool Selection | Verifies the agent used the expected tool (e.g., calculator) | Agent capability assessment |
| Correctness | Checks if response contains expected facts | Accuracy measurement |
| Relevance to Query | Evaluates if response directly addresses the user's question | Response quality assessment |

---

## Next Steps

To continue your journey with Strands Agents and SageMaker AI:

- **[Amazon SageMaker AI Documentation](https://docs.aws.amazon.com/sagemaker/)** - Learn more about model deployment and management
- **[Strands Agents SDK](https://github.com/strands-agents/strands-agents)** - Explore how to build and customize AI agents
- **[Strands Evaluation Guide](https://strandsagents.com/latest/documentation/docs/user-guide/observability-evaluation/evaluation/)** - Deep dive into agent evaluation
- **[SageMaker AI MLflow](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow.html)** - Dive deeper into agent observability using SageMaker AI MLflow App
- **[MLflow GenAI Evaluation](https://mlflow.org/docs/latest/genai/eval-monitor.html)** - Learn about MLflow's evaluation capabilities

---

**Happy building!** 🚀